# 5: Caption postprocessing
The phrase grounding models tend to use the word "group" as a label, rather than the noun following it. This error is fixed by checking if the word "group" is followed by a noun, and if it is, the label is changed to the noun. 


In [1]:
import pickle
import sys
from pathlib import Path

import pandas as pd

import src.GLIP.maskrcnn_benchmark as maskrcnn_benchmark

sys.modules['maskrcnn_benchmark'] = maskrcnn_benchmark


In [ ]:
phrase_grounding_selected_folder = '../data/phrase_grounding_selected/'
dataset_for_phrase_grounding = '../data/dataset_for_phrase_grounding/'


In [ ]:
# iterate over files in
# that directory
files = Path(phrase_grounding_selected_folder).glob('*.p')

pickle_filename = []
for file in files:
    pickle_filename.append(str(file))
pickle_filename

In [ ]:
dataset = pd.DataFrame()
for pickle_file in pickle_filename:
    dataset = dataset.append(pickle.load(open(pickle_file, 'rb')), ignore_index=True)

In [ ]:
dataset

In [ ]:
dataset['conf'] = dataset.results.apply(lambda x: x[0])
dataset['bbox'] = dataset.results.apply(lambda x: x[1])
dataset['labels'] = dataset.results.apply(lambda x: x[2])
dataset = dataset.drop(columns='results')

In [ ]:
expr_dataset = pickle.load(open(dataset_for_phrase_grounding + 'dataset.p', 'rb'))
expr_dataset = pd.DataFrame(expr_dataset)
expr_dataset['caption_raw'] = expr_dataset.caption.apply(lambda x: x['raw'])
expr_dataset['caption_preprocessed'] = expr_dataset.caption.apply(lambda x: x['preprocessed'])
expr_dataset['title_raw'] = expr_dataset.title.apply(lambda x: x['raw'])
expr_dataset['title_preprocessed'] = expr_dataset.title.apply(lambda x: x['preprocessed'])
expr_dataset = expr_dataset.drop(columns=['caption', 'title'])
expr_dataset.head()

In [ ]:
dataset = pd.merge(dataset, expr_dataset, left_on='filename', right_on='filename', how='left')

In [ ]:
def remove_group(elem):
    if ('a group' in elem['labels']) or ('group' in elem['labels']):
        if elem['expr'] == 'caption':
            expression = elem['caption_preprocessed']
        else:
            expression = elem['title_preprocessed']
        expression = expression.split(' ')
        try:
            word = expression[expression.index('of') + 1]
            if word == 'young':
                word = word + ' ' + expression[expression.index('of') + 2]
            return [word if label == 'a group' else label for label in elem['labels']]
        except:
            return elem['labels']

    else:
        return elem['labels']

In [ ]:
dataset['labels'] = dataset.apply(lambda x: remove_group(x), axis=1)

In [ ]:
pickle.dump(dataset, open('../data/dataset_for_segmentation.p','wb'))